# Usage
We ultimately want to fill in manga ratings that are NA or null.


We are choosing SBERT (Sentence BERT) as opposed to TFIDF cosine similarity because we want to predict ratings based on synopsis's that are semantically similar. It uses transformer-based architectures, such as BERT, that understand context and relationships between words, enabling it to capture the meaning of a sentence as a whole. TFIDF won't be a good method to use because it primiarily checks for word occurance and frequency which doesn't capture contexts or semantics.

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer


c:\Users\tpngu\anaconda3\envs\manga-rec\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
df = pd.read_csv('dataset/modified_manga.csv')
# MODEL = 'all-mpnet-base-v2' # General-purpose, best for applications where accuracy is more important than speed.
MODEL = 'paraphrase-MiniLM-L6-v2' # Balanced Performance. Paraphrase identification, text similarity tasks.
# MODEL = 'paraphrase-mpnet-base-v2' # High-accuracy paraphrase identification and text similarity tasks. Slightly slower than MiniLM

In [3]:
# Assuming the column containing synopses is named 'synopsis'
synopses = df['synopsis'].tolist()

# Load a pre-trained Sentence-BERT model
model = SentenceTransformer(MODEL) 

# Compute embeddings for each synopsis
embeddings = model.encode(synopses, convert_to_tensor=True)

c:\Users\tpngu\anaconda3\envs\manga-rec\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tpngu\.cache\huggingface\hub\models--sentence-transformers--paraphrase-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
# Now, 'embeddings' is a tensor containing the sentence embeddings for each synopsis
# You can convert it to a numpy array if needed:
embeddings_np = embeddings.cpu().numpy()

# Convert each embedding to a comma-separated string
embeddings_str = [','.join(map(str, embedding)) for embedding in embeddings_np]

# Add the embeddings back to the dataframe
df['embeddings'] = embeddings_str

# # If you want to add the embeddings back to the dataframe and save it
# df['embeddings'] = list(embeddings_np)

fname = 'dataset/manga_dataset_with_embeddings_' + MODEL + '.csv'

df.to_csv(fname, index=False)